In [25]:
import os
import re

import openai
from PIL import Image
from vif.prompts.oracle_prompts import ORACLE_CODE_EXPR_SYSTEM_PROMPT, ORACLE_CODE_PROMPT, ORACLE_PROPERTY_USAGE_PROMPT
from vif.utils.image_utils import encode_image

key = os.environ.get("OPENROUTER_API_KEY")
base_image = Image.open("notebooks/verifier_execution/debug_image.png")
encoded_image = encode_image(base_image)

In [26]:
instruction = 'Color the nodes of the graph in the color of the zone it is in. If a node is contained in multiple zones,  color it with the color of each zone (half of each color if in two zones, a third if in three, etc).'
model = "qwen/qwen3-vl-8b-instruct"

In [33]:
import json

oracle_code_instructions = ORACLE_CODE_PROMPT.format(instruction=instruction)
messages = [
    {
        "role": "system",
        "content": ORACLE_CODE_EXPR_SYSTEM_PROMPT
                   + "\n"
                   + ORACLE_PROPERTY_USAGE_PROMPT,
    },
    {
        "role": "user",
        "content": (
            oracle_code_instructions
            + "\n\n"
            + f"[IMAGE: data:image/png;base64,{encoded_image}]"
        ),
    },
]




In [46]:
import requests
import json

url = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {key}",
    "Content-Type": "application/json",
}

payload = {
    "model": model,
    "messages": messages,
    "stream": True,
    "temperature": 0.3,
}

buffer = ""

with requests.post(url, headers=headers, json=payload, stream=True) as r:
    r.raise_for_status()
    for chunk in r.iter_content(chunk_size=1024, decode_unicode=True):
        buffer += chunk

        while True:
            line_end = buffer.find("\n")
            if line_end == -1:
                break

            line = buffer[:line_end].strip()
            buffer = buffer[line_end + 1:]

            if not line.startswith("data: "):
                continue

            data = line[6:]

            if data == "[DONE]":
                break

            try:
                obj = json.loads(data)
            except json.JSONDecodeError:
                continue

            delta = obj["choices"][0]["delta"]
            content = delta.get("content")
            if content:
                print(content, end="", flush=True)


```python
def test_valid_customization() -> bool:
    return visual_property("The nodes of the graph are colored according to the zone they are in. If a node is contained in multiple zones, it is colored with the color of each zone (half of each color if in two zones, a third if in three, etc.)")
```